In [101]:
import warnings
import pandas as pd
import math
import numpy as np
import os
from multiprocessing import cpu_count
import lightgbm as lgb
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from feature_engine.encoding import MeanEncoder
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [77]:
def get_version_dir(version_name):
    dir_path = os.path.join(os.getcwd(), version_name)
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    return dir_path

In [2]:
def load_data(version=None):
    if version is None:
        return pd.read_csv('train_public.csv'), pd.read_csv('train_internet.csv'), pd.read_csv('test_public.csv')
    else:
        dir_path = get_version_dir(version)
        return pd.read_csv(os.path.join('train_public.csv')), pd.read_csv(os.path.join('train_internet.csv')), pd.read_csv(os.path.join('test_public.csv'))

In [3]:
train_public, train_internet, test_public = load_data()

In [4]:
train_public.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,censor_status,issue_date,use,post_code,region,debt_loan_ratio,del_in_18month,scoring_low,scoring_high,known_outstanding_loan,known_dero,pub_dero_bankrup,recircle_b,recircle_u,initial_list_status,app_type,earlies_credit_mon,title,policy_code,f0,f1,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon,isDefault
0,1040418,240418,31818.18182,3,11.466,1174.91,C,政府机构,金融业,3 years,0,1,2016/10/1,2,193,13,2.430000,0,556.363636,649.090909,3,0,0.0,7734.230769,91.800000,0,0,1-Dec,5,1,1.0,0.0,4.0,5.0,4.0,3,9927,0.0,0
1,1025197,225197,28000.00000,5,16.841,670.69,C,政府机构,金融业,10+ years,0,2,2013/6/1,0,491,30,11.004545,1,715.000000,893.750000,3,0,0.0,31329.000000,54.800000,1,0,Apr-90,40642,1,7.0,0.0,4.0,45.0,22.0,0,0,0.0,0
2,1009360,209360,17272.72727,3,8.900,603.32,A,政府机构,公共服务、社会组织,10+ years,1,0,2014/1/1,4,459,8,6.409091,0,774.545455,903.636364,5,0,0.0,18514.000000,57.692308,1,0,Oct-91,154,1,6.0,0.0,6.0,28.0,19.0,0,0,0.0,0
3,1039708,239708,20000.00000,3,4.788,602.30,A,世界五百强,文化和体育业,6 years,0,1,2015/7/1,0,157,8,9.204545,0,750.000000,875.000000,3,0,0.0,20707.000000,42.600000,0,0,1-Jun,0,1,5.0,0.0,10.0,15.0,9.0,0,0,0.0,0
4,1027483,227483,15272.72727,3,12.790,470.31,C,政府机构,信息传输、软件和信息技术服务业,< 1 year,2,1,2016/7/1,0,38,21,15.578182,0,609.090909,710.606061,15,0,0.0,14016.153850,30.461538,0,0,2-May,0,1,10.0,0.0,6.0,15.0,4.0,0,0,0.0,0


In [5]:
train_internet.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,sub_class,work_type,employer_type,industry,work_year,house_exist,house_loan_status,censor_status,marriage,offsprings,issue_date,use,post_code,region,debt_loan_ratio,del_in_18month,scoring_low,scoring_high,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,recircle_b,recircle_u,initial_list_status,earlies_credit_mon,title,policy_code,f0,f1,f2,f3,f4,f5,is_default
0,119262,0,12000.0,5,11.53,264.10,B,B5,职员,普通企业,采矿业,NaN,0,0,2,0,0,2015-06-01,0,814.0,4,5.07,1.0,670.0,674.0,1.0,0,0,0.0,3855.0,23.1,0,Mar-1984,0.0,1.0,1.0,0.0,8.0,17.0,8.0,1.0,1
1,369815,1,8000.0,3,13.98,273.35,C,C3,其他,普通企业,国际组织,10+ years,0,1,2,1,3,2010-10-01,2,240.0,21,15.04,0.0,725.0,729.0,0.0,0,0,0.0,118632.0,99.9,1,Jan-1992,94.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,787833,2,20000.0,5,17.99,507.76,D,D2,工人,上市企业,信息传输、软件和信息技术服务业,10+ years,0,0,1,0,0,2016-08-01,0,164.0,20,17.38,1.0,675.0,679.0,0.0,0,0,0.0,15670.0,72.5,0,Oct-1996,0.0,1.0,6.0,0.0,10.0,8.0,3.0,0.0,0
3,671675,3,10700.0,3,10.16,346.07,B,B1,职员,普通企业,电力、热力生产供应业,2 years,2,0,2,0,0,2013-05-01,4,48.0,10,27.87,0.0,710.0,714.0,0.0,0,0,0.0,18859.0,78.6,0,Jul-2000,41646.0,1.0,3.0,0.0,4.0,11.0,6.0,0.0,0
4,245160,4,8000.0,3,8.24,251.58,B,B1,其他,政府机构,金融业,5 years,1,2,0,0,0,2017-04-01,4,122.0,9,3.47,0.0,660.0,664.0,0.0,0,0,0.0,8337.0,67.8,1,Mar-2000,4.0,1.0,3.0,0.0,8.0,6.0,4.0,1.0,0


In [6]:
test_public.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,work_year,house_exist,censor_status,issue_date,use,post_code,region,debt_loan_ratio,del_in_18month,scoring_low,scoring_high,known_outstanding_loan,known_dero,pub_dero_bankrup,recircle_b,recircle_u,initial_list_status,app_type,earlies_credit_mon,title,policy_code,f0,f1,f2,f3,f4,early_return,early_return_amount,early_return_amount_3mon
0,1000575,200575,2890.909091,3,10.791,88.01,B,幼教与中小学校,住宿和餐饮业,5 years,0,1,2017/12/1,0,314,0,23.040000,0,745.000000,869.166667,7,0,0.0,8647.692308,31.846154,1,0,3-Mar,0,1,2.0,0.0,15.0,5.0,4.0,3,773,89.192308
1,1028125,228125,7272.727273,3,9.990,258.10,B,普通企业,批发和零售业,10+ years,1,1,2015/7/1,5,29,19,27.754545,0,681.818182,738.636364,24,0,0.0,9406.153846,18.276923,0,0,Dec-99,6,1,8.0,0.0,8.0,29.0,14.0,1,1894,218.538462
2,1010694,210694,26295.454550,3,15.763,764.03,C,普通企业,住宿和餐饮业,10+ years,0,2,2013/4/1,0,488,24,25.494545,1,758.181818,947.727273,11,0,0.0,26414.769230,62.300000,1,0,Apr-99,268,1,6.0,0.0,4.0,10.0,6.0,1,5670,1221.230769
3,1026712,226712,22690.909090,5,19.305,524.30,D,普通企业,采矿业,10+ years,0,2,2017/12/1,0,489,30,10.620000,0,572.727273,620.454545,8,0,0.0,1198.000000,7.700000,0,0,Jul-00,0,1,4.0,0.0,12.0,10.0,8.0,2,4800,443.076923
4,1002895,202895,14545.454550,3,7.139,490.32,A,世界五百强,金融业,1 year,0,0,2016/6/1,2,418,45,6.610909,0,638.181818,691.363636,15,0,0.0,3920.000000,8.830769,1,0,7-May,5,1,4.0,0.0,7.0,14.0,9.0,0,3516,649.107692


In [7]:
train_public.rename(columns={'isDefault': 'is_default'}, inplace=True)

In [8]:
# 最后再看看有哪些列不一样的
same_columns = list(train_public.columns.intersection(set(train_internet.columns)))

In [9]:
train_public.columns.difference(set(same_columns))

Index(['app_type', 'known_dero', 'known_outstanding_loan'], dtype='object')

In [10]:
train_internet.columns.difference(set(same_columns))

Index(['f5', 'house_loan_status', 'marriage', 'offsprings', 'sub_class',
       'work_type'],
      dtype='object')

In [11]:
# 跟 train_public 一样，
test_public.columns.difference(set(same_columns))

Index(['app_type', 'known_dero', 'known_outstanding_loan'], dtype='object')

In [12]:
# 可以internet为主做特征判断
len(train_public), len(train_internet), len(test_public)

(10000, 750000, 5000)

# 初看特征
填充缺失值

In [13]:
# 看一些哪些列要填充
train_internet.isna().sum()

loan_id                         0
user_id                         0
total_loan                      0
year_of_loan                    0
interest                        0
monthly_payment                 0
class                           0
sub_class                       0
work_type                       0
employer_type                   0
industry                        0
work_year                   43847
house_exist                     0
house_loan_status               0
censor_status                   0
marriage                        0
offsprings                      0
issue_date                      0
use                             0
post_code                       1
region                          0
debt_loan_ratio               230
del_in_18month                  0
scoring_low                     0
scoring_high                    0
pub_dero_bankrup              375
early_return                    0
early_return_amount             0
early_return_amount_3mon        0
recircle_b    

In [14]:
# 看一些哪些列要填充
train_public.isna().sum()

loan_id                       0
user_id                       0
total_loan                    0
year_of_loan                  0
interest                      0
monthly_payment               0
class                         0
employer_type                 0
industry                      0
work_year                   622
house_exist                   0
censor_status                 0
issue_date                    0
use                           0
post_code                     0
region                        0
debt_loan_ratio               0
del_in_18month                0
scoring_low                   0
scoring_high                  0
known_outstanding_loan        0
known_dero                    0
pub_dero_bankrup              7
recircle_b                    0
recircle_u                    0
initial_list_status           0
app_type                      0
earlies_credit_mon            0
title                         0
policy_code                   0
f0                          498
f1      

In [15]:
test_public.isna().sum()

loan_id                       0
user_id                       0
total_loan                    0
year_of_loan                  0
interest                      0
monthly_payment               0
class                         0
employer_type                 0
industry                      0
work_year                   278
house_exist                   0
censor_status                 0
issue_date                    0
use                           0
post_code                     0
region                        0
debt_loan_ratio               0
del_in_18month                0
scoring_low                   0
scoring_high                  0
known_outstanding_loan        0
known_dero                    0
pub_dero_bankrup              4
recircle_b                    0
recircle_u                    0
initial_list_status           0
app_type                      0
earlies_credit_mon            0
title                         0
policy_code                   0
f0                          259
f1      

In [16]:
all_dataframe = [train_public, train_internet, test_public]

In [17]:
def group_agg(data, groupby, agg_field):
    return data.groupby([groupby]).agg(max_value=(agg_field, 'max'), min_value=(agg_field, 'min'), agg_value=(agg_field, 'mean'))

In [18]:
# 贷款上限没有区别，平均值在上升
group_agg(train_internet, 'class', 'total_loan')

,max_value,min_value,agg_value
class,,,
A,40000.0,500.0,13863.324674
B,40000.0,500.0,13252.606059
C,40000.0,700.0,14185.076498
D,40000.0,1000.0,15296.709322
E,40000.0,1000.0,17576.450378
F,40000.0,1000.0,19114.208331
G,40000.0,1000.0,20612.438082


In [19]:
# 每月还款能力有提升
group_agg(train_internet, 'class', 'monthly_payment')

,max_value,min_value,agg_value
class,,,
A,1268.46,15.69,422.841167
B,1342.57,16.31,404.145465
C,1406.68,22.59,425.347520
D,1517.09,23.51,470.493745
E,1618.03,35.32,528.030922
F,1714.54,27.82,586.063088
G,1715.42,40.19,662.203600


In [20]:
# sub_class 同理
group_agg(train_internet, 'sub_class', 'monthly_payment')

,max_value,min_value,agg_value
sub_class,,,
A1,1225.24,22.51,422.224041
A2,1236.37,30.33,407.767910
A3,1245.91,19.87,419.180399
A4,1256.97,15.69,430.840854
A5,1268.46,20.22,427.729581
B1,1296.90,31.42,396.398542
B2,1304.26,21.25,405.668219
B3,1316.95,21.74,409.195314
B4,1324.76,32.10,405.585271


In [21]:
# 附一个平均值
train_public['sub_class'] = train_public['class'] + '3'

In [22]:
def fill_class_score(data):
    class_score = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}
    # 插空值 实际没有空值
    class_without_na = data['class'].fillna('C').astype('str')
    if 'sub_class' not in data.columns:
        sub_class_without_na = class_without_na + '3'
    else:
        # 插空值 实际没有空值
        sub_class_without_na = data['sub_class'].fillna('C3')
    data['class_score'] = sub_class_without_na.apply(lambda r: class_score[r[0:1]] + int(r[1:2]) * 0.2)

In [23]:
# 转换成分数
fill_class_score(train_public)
fill_class_score(train_internet)
fill_class_score(test_public)

In [24]:
# 工作年份越高借得越多
group_agg(train_internet, 'work_year', 'total_loan')

,max_value,min_value,agg_value
work_year,,,
1 year,40000.0,725.0,13497.980347
10+ years,40000.0,1000.0,15625.733066
2 years,40000.0,1000.0,13706.342389
3 years,40000.0,900.0,13870.826819
4 years,40000.0,1000.0,14055.518796
5 years,40000.0,1000.0,14127.187573
6 years,40000.0,1000.0,14358.016328
7 years,40000.0,1000.0,14596.507530
8 years,40000.0,1000.0,14716.324541


In [25]:
# 工作年份越高每月还得越多，可以假设class_score有办法对应到work_year
group_agg(train_internet, 'work_year', 'monthly_payment')

,max_value,min_value,agg_value
work_year,,,
1 year,1618.03,19.87,413.927120
10+ years,1715.42,21.74,469.676688
2 years,1566.59,28.69,419.670221
3 years,1691.28,23.17,424.730172
4 years,1566.80,30.12,429.041680
5 years,1598.26,20.22,430.675399
6 years,1647.03,25.28,436.338774
7 years,1566.80,21.25,442.354609
8 years,1714.54,30.46,445.511337


In [26]:
# 居然全是10年+
train_internet.groupby(['class_score', 'work_year'], as_index=False).agg(cnt=('work_year', 'count')).sort_values('cnt', ascending=False).groupby('class_score', as_index=True).first()

,work_year,cnt
class_score,,
0.2,10+ years,8603
0.4,10+ years,6968
0.6,10+ years,7238
0.8,10+ years,9629
1.0,10+ years,11916
1.2,10+ years,13470
1.4,10+ years,13957
1.6,10+ years,15055
1.8,10+ years,14902


In [27]:
# 大部分都是10年
train_internet.groupby(['work_year'], as_index=False).agg(cnt=('work_year', 'count'))

,work_year,cnt
0,1 year,49204
1,10+ years,246226
2,2 years,67987
3,3 years,60128
4,4 years,45037
5,5 years,47027
6,6 years,34910
7,7 years,33200
8,8 years,33857
9,9 years,28379


In [28]:
# 依赖 fill_class_score
def fill_work_year_num(data):
    # 缺失值填 10+ years
    work_year_withou_na = data['work_year'].fillna('10+ years').replace('< 1 year', '0 year').replace('10+ years', '10 years').apply(lambda r: int(r[0:r.index(' ')]))
    # 10年以上的加上class_score
    data['work_year_num'] = work_year_withou_na + (work_year_withou_na >= 10) * data['class_score']

In [29]:
fill_work_year_num(train_public)
fill_work_year_num(train_internet)
fill_work_year_num(test_public)

In [30]:
# 验证状态越高借得越多
group_agg(train_internet, 'censor_status', 'total_loan')

,max_value,min_value,agg_value
censor_status,,,
0,40000.0,500.0,11686.319937
1,40000.0,1000.0,14733.717221
2,40000.0,1000.0,16675.890914


In [31]:
# 验证状态越高还得越多，可保留数值
group_agg(train_internet, 'censor_status', 'monthly_payment')

,max_value,min_value,agg_value
censor_status,,,
0,1714.54,15.69,356.725685
1,1714.54,23.51,445.140440
2,1715.42,20.22,507.952148


In [32]:
# 处理 网络贷款发放的日期
def fill_issue_date(data):
    df_issue_date = pd.to_datetime(data['issue_date'])
    data['issue_date_y'] = df_issue_date.dt.year
    data['issue_date_m'] = df_issue_date.dt.month
    data['issue_date_d'] = df_issue_date.dt.day
    data['issue_date_dow'] = df_issue_date.dt.dayofweek

In [33]:
fill_issue_date(train_public)
fill_issue_date(train_internet)
fill_issue_date(test_public)

In [34]:
# 用途 数值与贷款金额无关
group_agg(train_internet, 'use', 'total_loan')

,max_value,min_value,agg_value
use,,,
0,40000.0,700.0,15231.344548
1,40000.0,1000.0,15795.733708
2,40000.0,1000.0,14112.724967
3,40000.0,1000.0,11891.027938
4,40000.0,725.0,14800.446290
5,40000.0,500.0,9828.315739
6,40000.0,1000.0,15429.348894
7,40000.0,500.0,6263.941258
8,40000.0,1000.0,8882.279104


In [35]:
# 用途 数值与每月还款无关，可按类目处理
group_agg(train_internet, 'use', 'monthly_payment')

,max_value,min_value,agg_value
use,,,
0,1715.42,20.22,462.269784
1,1714.54,22.79,503.823068
2,1714.54,30.12,423.533535
3,1533.81,28.69,360.644698
4,1566.80,22.24,445.777151
5,1566.59,15.69,311.178595
6,1569.11,30.12,479.528730
7,1393.64,16.31,203.893296
8,1466.85,21.74,272.156490


In [36]:
# post_code 填一个出现最多的
train_internet['post_code'] = train_internet['post_code'].fillna(
    train_internet.groupby(['post_code'], as_index=False).agg(cnt=('post_code', 'count')).sort_values('cnt', ascending=False).iloc[0].post_code
).astype('str')

In [37]:
# debt_loan_ratio 债务收入比，随网贷等级升高
group_agg(train_internet, 'class_score', 'debt_loan_ratio')

,max_value,min_value,agg_value
class_score,,,
0.2,460.21,0.00,14.136915
0.4,173.06,0.00,15.178436
0.6,171.39,0.00,15.777530
0.8,999.00,0.00,15.998092
1.0,483.88,0.00,16.426370
1.2,831.97,0.00,16.689871
1.4,999.00,0.00,17.116708
1.6,999.00,-1.00,17.422321
1.8,999.00,0.00,17.673856


In [38]:
# 按 agg_decision_field 取平均值填充 fill_field 的缺失值
def fill_na_from_agg_map(data, agg_map, fill_field, agg_decision_field):
    data[fill_field] = data.apply(lambda r: r[fill_field] if not np.isnan(r[fill_field]) else agg_map[r[agg_decision_field]], axis=1)

In [39]:
#debt_loan_ratio 债务收入比，按网贷等级填平均值
debt_loan_ratio_agg_map = group_agg(train_internet, 'class_score', 'debt_loan_ratio').drop(['min_value', 'max_value'], axis=1).to_dict()['agg_value']
fill_na_from_agg_map(train_public, debt_loan_ratio_agg_map, 'debt_loan_ratio', 'class_score')
fill_na_from_agg_map(train_internet, debt_loan_ratio_agg_map, 'debt_loan_ratio', 'class_score')
fill_na_from_agg_map(test_public, debt_loan_ratio_agg_map, 'debt_loan_ratio', 'class_score')

In [40]:
# pub_dero_bankrup 公开记录清除的数量，随网贷等级升高
group_agg(train_internet, 'class_score', 'pub_dero_bankrup')

,max_value,min_value,agg_value
class_score,,,
0.2,4.0,0.0,0.029882
0.4,4.0,0.0,0.045409
0.6,7.0,0.0,0.064326
0.8,5.0,0.0,0.074651
1.0,5.0,0.0,0.086498
1.2,12.0,0.0,0.140821
1.4,8.0,0.0,0.135696
1.6,6.0,0.0,0.135058
1.8,7.0,0.0,0.139047


In [41]:
# pub_dero_bankrup 公开记录清除的数量，按网贷等级取平均值填充缺失值
pub_dero_bankrup_agg_map = group_agg(train_internet, 'class_score', 'pub_dero_bankrup').drop(['min_value', 'max_value'], axis=1).to_dict()['agg_value']
fill_na_from_agg_map(train_public, pub_dero_bankrup_agg_map, 'pub_dero_bankrup', 'class_score')
fill_na_from_agg_map(train_internet, pub_dero_bankrup_agg_map, 'pub_dero_bankrup', 'class_score')
fill_na_from_agg_map(test_public, pub_dero_bankrup_agg_map, 'pub_dero_bankrup', 'class_score')

In [42]:
# recircle_u 循环额度利用率，随网贷等级升高
group_agg(train_internet, 'class_score', 'recircle_u')

,max_value,min_value,agg_value
class_score,,,
0.2,101.4,0.0,29.794876
0.4,102.8,0.0,34.958633
0.6,124.7,0.0,38.000147
0.8,117.2,0.0,41.668422
1.0,146.1,0.0,44.998510
1.2,119.4,0.0,46.759399
1.4,162.0,0.0,49.429678
1.6,132.2,0.0,51.028573
1.8,892.3,0.0,52.035036


In [43]:
# recircle_u 循环额度利用率，按网贷等级取平均值填充缺失值
recircle_u_agg_map = group_agg(train_internet, 'class_score', 'recircle_u').drop(['min_value', 'max_value'], axis=1).to_dict()['agg_value']
fill_na_from_agg_map(train_public, recircle_u_agg_map, 'recircle_u', 'class_score')
fill_na_from_agg_map(train_internet, recircle_u_agg_map, 'recircle_u', 'class_score')
fill_na_from_agg_map(test_public, recircle_u_agg_map, 'recircle_u', 'class_score')

In [44]:
# 借款人在信用评分系统所属的得分区间 可合并，或者只用一个分数，选择只用下限
train_internet.groupby(['scoring_low', 'scoring_high']).agg(cnt=('scoring_low', 'count'))

,,cnt
scoring_low,scoring_high,
630.0,634.0,1
660.0,664.0,67466
665.0,669.0,65241
670.0,674.0,65559
675.0,679.0,58196
680.0,684.0,57375
685.0,689.0,50286
690.0,694.0,48804
695.0,699.0,44107


In [45]:
def format_earliest_credit_month(val):
    if isinstance(val, pd._libs.tslibs.timestamps.Timestamp):
        return val
    fd = re.search('(\d+-)', val)
    if fd is None:
        return '1-'+val
    return val + '-01'

In [131]:
def handle_earlies_credit_mon(df):
    time_max = pd.to_datetime('1-Dec-21')
    df['earlies_credit_mon'] = pd.to_datetime(df['earlies_credit_mon'].map(format_earliest_credit_month))
    too_large_idx = df['earlies_credit_mon'] > time_max
    df.loc[too_large_idx, 'earlies_credit_mon' ] = df.loc[too_large_idx, 'earlies_credit_mon' ] + pd.offsets.DateOffset(years=-100)
    df['earliest_credit_year'] = df['earlies_credit_mon'].dt.year
    df['earliest_credit_month'] = df['earlies_credit_mon'].dt.month
    #df['earliest_credit_ym'] = df.apply(lambda r: '%d%02d' % (r['earliest_credit_year'], r['earliest_credit_month']), axis=1)

In [132]:
handle_earlies_credit_mon(train_public)
handle_earlies_credit_mon(train_internet)
handle_earlies_credit_mon(test_public)

In [48]:
# 网络贷款的初始列表状态 自身就是01
group_agg(train_internet, 'initial_list_status', 'total_loan')

,max_value,min_value,agg_value
initial_list_status,,,
0,40000.0,1000.0,14891.729484
1,40000.0,500.0,13759.545630


In [49]:
train_public['post_code'] = train_public['post_code'].astype('float').astype('int')
train_internet['post_code'] = train_internet['post_code'].astype('float').astype('int')
test_public['post_code'] = test_public['post_code'].astype('float').astype('int')

In [50]:
# title 什么玩意 先丢掉
len(train_internet['title'].unique())

37499

In [51]:
# 一样的 丢掉
train_public['policy_code'].unique(), train_internet['policy_code'].unique()

(array([1]), array([1.]))

In [52]:
# f0 ~ f4 填充缺失值
for i in range(5):
    f_field = 'f%d' % i
    f_agg_map = group_agg(train_internet, 'class_score', f_field).drop(['min_value', 'max_value'], axis=1).to_dict()['agg_value']
    fill_na_from_agg_map(train_public, f_agg_map, f_field, 'class_score')
    fill_na_from_agg_map(train_internet, f_agg_map, f_field, 'class_score')
    fill_na_from_agg_map(test_public, f_agg_map, f_field, 'class_score')

In [53]:
# public的user_id居然都能在internet中找到
len(set(train_public['user_id'].unique().tolist()).intersection(set(train_internet['user_id'].unique().tolist())))

10000

In [54]:
# 看上去并不是同一个 user_id，那就没办法从 internet 数据集关联更多字段出来了
train_public.iloc[0]['employer_type'], train_internet.loc[train_internet['user_id'] == train_public.iloc[0]['user_id']]['employer_type']

('政府机构', 240418    上市企业
 Name: employer_type, dtype: object)

In [127]:
train_internet.head()

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,sub_class,work_type,employer_type,industry,work_year,house_exist,house_loan_status,censor_status,marriage,offsprings,issue_date,use,post_code,region,debt_loan_ratio,del_in_18month,scoring_low,scoring_high,pub_dero_bankrup,early_return,early_return_amount,early_return_amount_3mon,recircle_b,recircle_u,initial_list_status,earlies_credit_mon,title,policy_code,f0,f1,f2,f3,f4,f5,is_default,class_score,work_year_num,issue_date_y,issue_date_m,issue_date_d,issue_date_dow,earliest_credit_year,earliest_credit_month,total_loan_avg,monthly_payment_avg,total_loan_diff,monthly_payment_diff
0,119262,0,12000.0,5,11.53,264.10,B,B5,职员,普通企业,采矿业,NaN,0,0,2,0,0,2015-06-01,0,814,4,5.07,1.0,670.0,674.0,1.0,0,0,0.0,3855.0,23.1,0,1984-03-01,0.0,1.0,1.0,0.0,8.0,17.0,8.0,1.0,1,2.0,12.0,2015,6,1,0,1984,3,12901.03167,403.451382,-901.03167,-139.351382
1,438556,933,15000.0,3,11.49,494.57,B,B5,其他,普通企业,采矿业,< 1 year,0,1,2,1,0,2016-10-01,4,512,21,23.52,0.0,675.0,679.0,0.0,0,0,0.0,71518.0,88.1,0,2000-04-01,4.0,1.0,5.0,0.0,9.0,12.0,8.0,2.0,0,2.0,0.0,2016,10,1,5,2000,4,12901.03167,403.451382,2098.96833,91.118618
2,67067,2242,16000.0,3,11.47,527.39,B,B5,其他,普通企业,采矿业,7 years,0,2,2,1,3,2016-03-01,0,328,3,34.32,0.0,740.0,744.0,0.0,0,0,0.0,23278.0,23.2,0,1987-02-01,0.0,1.0,12.0,0.0,10.0,43.0,26.0,0.0,0,2.0,7.0,2016,3,1,1,1987,2,12901.03167,403.451382,3098.96833,123.938618
3,358213,5389,3200.0,3,11.53,105.57,B,B5,其他,普通企业,采矿业,10+ years,1,0,0,1,3,2015-05-01,7,368,3,31.87,1.0,665.0,669.0,0.0,0,0,0.0,33543.0,59.0,1,2002-12-01,8.0,1.0,16.0,0.0,5.0,19.0,11.0,1.0,0,2.0,12.0,2015,5,1,4,2002,12,12901.03167,403.451382,-9701.03167,-297.881382
4,497535,7047,6400.0,3,11.49,211.02,B,B5,公务员,普通企业,采矿业,10+ years,1,1,2,0,0,2016-11-01,4,237,32,29.67,2.0,730.0,734.0,0.0,0,0,0.0,13136.0,28.2,1,1978-06-01,4.0,1.0,7.0,0.0,8.0,28.0,18.0,0.0,0,2.0,12.0,2016,11,1,1,1978,6,12901.03167,403.451382,-6501.03167,-192.431382


# EDA

In [139]:
def histplot(df, x, target='is_default'):
    sns.histplot(data=df, x=x, hue=target, multiple="stack")

In [143]:
def displot(df, x, y, target='is_default'):
    sns.displot(data=df, x=x, y=y, hue=target, kind="kde")

# 特征工程

## 数值字段分桶

In [155]:
to_disc_columns = ['total_loan', 'interest', 'monthly_payment', 'debt_loan_ratio', 'del_in_18month', 'pub_dero_bankrup', 'early_return', 
                   'early_return_amount', 'early_return_amount_3mon', 'recircle_b', 'recircle_u', 'f0', 'f1', 'f2', 'f3', 'f4']

In [163]:
from feature_engine.discretisation import EqualFrequencyDiscretiser
disc = EqualFrequencyDiscretiser(q=10, variables=to_disc_columns)
disc.fit(train_internet[to_disc_columns])

EqualFrequencyDiscretiser(variables=['total_loan', 'interest',
                                     'monthly_payment', 'debt_loan_ratio',
                                     'del_in_18month', 'pub_dero_bankrup',
                                     'early_return', 'early_return_amount',
                                     'early_return_amount_3mon', 'recircle_b',
                                     'recircle_u', 'f0', 'f1', 'f2', 'f3',
                                     'f4'])

In [168]:
for c in to_disc_columns:
    c_suf = '%s_disc' % c
    if c_suf in train_public:
        train_public.drop([c_suf], axis=1, inplace=True)
    if c_suf in train_internet:
        train_internet.drop([c_suf], axis=1, inplace=True)
    if c_suf in test_public:
        test_public.drop([c_suf], axis=1, inplace=True)
       
train_public = train_public.merge(disc.transform(train_public[to_disc_columns]), left_index=True, right_index=True, suffixes=('', '_disc'))
train_internet = train_internet.merge(disc.transform(train_internet[to_disc_columns]), left_index=True, right_index=True, suffixes=('', '_disc'))
test_public = test_public.merge(disc.transform(test_public[to_disc_columns]), left_index=True, right_index=True, suffixes=('', '_disc'))

## 组合字段找违约率分布较散的

In [175]:
to_group_columns = list(train_public.columns.intersection(set(train_internet.columns)))
for c in ['loan_id', 'user_id', 'class', 'work_year', 'issue_date', 'scoring_high', 'earlies_credit_mon', 'title', 'policy_code', 'is_default', 'sub_class']:
    to_group_columns.remove(c)
for c in to_disc_columns:
    to_group_columns.remove(c)

In [180]:
two_dimension_rate_column = []
for c in two_dimension_rate_column:
    if c_suf in train_public:
        train_public.drop([c_suf], axis=1, inplace=True)
    if c_suf in train_internet:
        train_internet.drop([c_suf], axis=1, inplace=True)
    if c_suf in test_public:
        test_public.drop([c_suf], axis=1, inplace=True)

for i in range(len(to_group_columns)):
    for j in range(len(to_group_columns)):
        if j <= i:
            continue
        c1 = to_group_columns[i]
        c2 = to_group_columns[j]
        agg_df = train_internet.groupby([c1, c2], as_index=False).agg(one=('is_default', 'sum'), cnt=('is_default', 'count'))
        rate_field = '%s_%s_rate' % (c1, c2)
        agg_df[rate_field] = agg_df['one'] / agg_df['cnt']
        if agg_df[rate_field].kurt() < -1:
            two_dimension_rate_column.append(rate_field)
            if rate_field in train_public:
                train_public.drop([rate_field], axis=1, inplace=True)
            if rate_field in train_internet:
                train_internet.drop([rate_field], axis=1, inplace=True)
            if rate_field in test_public:
                test_public.drop([rate_field], axis=1, inplace=True)
            train_public = train_public.merge(agg_df[[c1, c2, rate_field]], on=[c1, c2])
            train_internet = train_internet.merge(agg_df[[c1, c2, rate_field]], on=[c1, c2])
            test_public = test_public.merge(agg_df[[c1, c2, rate_field]], on=[c1, c2])

In [182]:
len(train_internet.columns)

178

In [55]:
# 分组平均值
user_group = ['class_score', 'employer_type', 'industry']
avg_columns = ['total_loan', 'monthly_payment']
user_group_df = train_internet.groupby(user_group, as_index=False).mean()[user_group + avg_columns]

In [56]:
for c in avg_columns:
    fields_to_drop = ['%s_avg' % c, '%s_diff' % c]
    train_public.drop([ctd for ctd in fields_to_drop if ctd in train_public.columns], axis=1, inplace=True)
    train_internet.drop([ctd for ctd in fields_to_drop if ctd in train_internet.columns], axis=1, inplace=True)
    test_public.drop([ctd for ctd in fields_to_drop if ctd in test_public.columns], axis=1, inplace=True)
    
train_public = train_public.merge(user_group_df, on=user_group, suffixes=('', '_avg'))
train_internet = train_internet.merge(user_group_df, on=user_group, suffixes=('', '_avg'))
test_public = test_public.merge(user_group_df, on=user_group, suffixes=('', '_avg'))

for c in avg_columns:
    train_public['%s_diff' % c] = train_public[c] - train_public['%s_avg' % c]
    train_internet['%s_diff' % c] = train_internet[c] - train_internet['%s_avg' % c]
    test_public['%s_diff' % c] = test_public[c] - test_public['%s_avg' % c]

In [185]:
def save_data(version, train_public, train_internet, test_public):
    dir_path = get_version_dir(version)
    train_public.to_csv(os.path.join(dir_path, 'train_public.csv'), index=False)
    train_internet.to_csv(os.path.join(dir_path, 'train_internet.csv'), index=False)
    test_public.to_csv(os.path.join(dir_path, 'test_public.csv'), index=False)

In [183]:
save_data('v2', train_public, train_internet, test_public)

In [44]:
train_public, train_internet, test_public = load_data('v1')

In [62]:
# 不能标准化的字段
not_std_columns = ['employer_type', 'industry', 'use', 'issue_date_y', 'issue_date_m', 'post_code', 'region', 'earliest_credit_year', 'earliest_credit_month']

In [102]:
not_std_columns_cnt = {}
onehot_columns = []
mean_label_columns = []
for c in not_std_columns:
    not_std_columns_cnt[c] = len(train_internet[c].unique())
    if not_std_columns_cnt[c] < 1:
        onehot_columns.append(c)
    else:
        mean_label_columns.append(c)
not_std_columns_cnt, onehot_columns, mean_label_columns

({'employer_type': 6,
  'industry': 14,
  'use': 14,
  'issue_date_y': 12,
  'issue_date_m': 12,
  'post_code': 930,
  'region': 51,
  'earliest_credit_year': 68,
  'earliest_credit_month': 12},
 [],
 ['employer_type',
  'industry',
  'use',
  'issue_date_y',
  'issue_date_m',
  'post_code',
  'region',
  'earliest_credit_year',
  'earliest_credit_month'])

In [103]:
# 直接保留的字段
keep_columns = ['initial_list_status']

In [104]:
# 可以丢掉的字段
drop_columns = ['loan_id', 'user_id', 'class', 'sub_class', 'work_type', 'work_year', 'house_loan_status', 'offsprings', 'marriage', 'issue_date',
                'issue_date_d', 'issue_date_dow', 'scoring_high', 'earlies_credit_mon', 'title', 'policy_code', 'f5', 'app_type', 'known_dero', 'known_outstanding_loan'] \
                + ['%s_avg' % c for c in avg_columns]
                + ['%s_disc' % c for c in disc_columns]

In [105]:
# 标准化的字段
std_columns = ['total_loan', 'year_of_loan', 'interest', 'monthly_payment', 'class_score', 'work_year_num', 'house_exist', 'censor_status', 'debt_loan_ratio',
               'del_in_18month', 'scoring_low', 'pub_dero_bankrup', 'early_return', 'early_return_amount', 'early_return_amount_3mon', 'recircle_b', 
               'recircle_u', 'f0', 'f1', 'f2', 'f3', 'f4'] \
                + ['%s_diff' % c for c in avg_columns]

In [186]:
final_same_columns = list(train_public.columns.intersection(set(train_internet.columns)))

In [187]:
final_data = pd.concat([train_public[final_same_columns].copy(), train_internet[final_same_columns].copy()], ignore_index=True)
test_data = test_public.copy()

In [188]:
train_y = final_data['is_default']
final_data.drop(['is_default'], axis=1, inplace=True)

In [189]:
for c in drop_columns:
    if c in final_data.columns:
        final_data.drop([c], axis=1, inplace=True)
    if c in test_data.columns:
        test_data.drop([c], axis=1, inplace=True)

In [190]:
# 不需要了
# for c in std_columns:
#     c_mean = final_data[c].astype(np.float32).mean()
#     c_std = final_data[c].astype(np.float32).std()
    
#     final_data[c] = (final_data[c].astype(np.float32) - c_mean) / c_std
#     test_data[c] = (test_data[c].astype(np.float32) - c_mean) / c_std

In [72]:
def get_onehot_encode_df(data, encoder, field):
    ohe_result = ohe.transform(data[[field]].astype('str'))
    return pd.DataFrame(ohe_result.toarray()).rename(lambda n: field + '_' + str(n), axis=1).astype('int')  

In [191]:
train_oh_dfs = [final_data]
test_oh_dfs = [test_data]
for c in onehot_columns:
    ohe = OneHotEncoder(handle_unknown='ignore').fit(final_data[[c]].astype('str'))
    train_oh_dfs.append(get_onehot_encode_df(final_data, ohe, c))
    test_oh_dfs.append(get_onehot_encode_df(test_data, ohe, c))
final_data = pd.concat(train_oh_dfs, axis=1)
test_data = pd.concat(test_oh_dfs, axis=1)
final_data.drop(onehot_columns, axis=1, inplace=True)
test_data.drop(onehot_columns, axis=1, inplace=True)
del train_oh_dfs
del test_oh_dfs

In [192]:
mean_label_encoder = MeanEncoder(variables=mean_label_columns)
final_data[mean_label_columns] = final_data[mean_label_columns].astype('category')
test_data[mean_label_columns] = test_data[mean_label_columns].astype('category')
final_data = mean_label_encoder.fit_transform(final_data, train_y)
test_data = mean_label_encoder.transform(test_data)

In [193]:
len(final_data.columns), len(test_data.columns)

(158, 158)

In [194]:
train_x = final_data
test_x = test_data[train_x.columns]

In [78]:
def save_dataset(version_name, train_x, train_y, test_x):
    dir_path = get_version_dir(version_name)
    train_x.to_csv(os.path.join(dir_path, 'train_x.csv'), index=False)
    train_y.to_csv(os.path.join(dir_path, 'train_y.csv'), index=False)
    test_x.to_csv(os.path.join(dir_path, 'test_x.csv'), index=False)

In [87]:
def save_submission(model, test_x):
    submission = pd.DataFrame({'id': test_public['loan_id'], 'isDefault': model.predict(test_x)})
    submission.to_csv('submission.csv', index = None)

In [90]:
def save_model(version_name, model):
    dir_path = get_version_dir(version_name)
    model.save_model(os.path.join(dir_path, 'model.txt'))

In [244]:
def train_and_save(version, train_x, train_y, test_x):
    final_param = {}
    final_param.update(param)
    del final_param['early_stopping_round']
    dataset = lgb.Dataset(train_x, label=train_y)
    model = lgb.train(final_param, dataset)
    train_predict_y = model.predict(train_x)
    test_predict_y = model.predict(test_x)
    
    # 剔除训练集中间部分的数据 加入测试集边缘数据
    train_idx = (train_predict_y < 0.45) | (train_predict_y > 0.55)
    test_idx = test_predict_y < 0.05
    train_x_2 = pd.concat([train_x.loc[train_idx].copy(), test_x.loc[test_idx].copy()], ignore_index=1)
    train_y_2 = pd.concat([train_y.loc[train_idx], pd.Series([0] * test_idx.sum())])
    dataset_2 = lgb.Dataset(train_x_2, label=train_y_2)
    model_2 = lgb.train(final_param, dataset_2)
    save_submission(model_2, test_x)
    #save_dataset(version, train_x_2, train_y_2, test_x)
    #save_model(version, model_2)
    return roc_auc_score(train_y, model.predict(train_x)), roc_auc_score(train_y_2, model_2.predict(train_x_2))

In [219]:
def avg_predict(model_list, x):
    y = None
    for model in model_list:
        if y is None:
            y = model.predict(x)
        else:
            y = y + model.predict(x)
    return y / len(model_list)

In [229]:
def kfold_train_and_save(version, train_x, train_y, test_x):
    skf = StratifiedKFold(n_splits=5, random_state=2021, shuffle=True)
    model_list = []
    auc_list = []
    for train_index, valid_index in skf.split(train_x, train_y):
        train_ds = lgb.Dataset(train_x.iloc[train_index], label=train_y.iloc[train_index])
        valid_ds = lgb.Dataset(train_x.iloc[valid_index], label=train_y.iloc[valid_index])
        model = lgb.train(param, train_ds, valid_sets=[valid_ds], verbose_eval=50)
        model_list.append(model)
    predict_y = avg_predict(model_list, train_x)
    return roc_auc_score(train_y, predict_y)

In [241]:
param = {
    'verbosity': -1,
    'objective': 'binary',
    'num_threads': cpu_count(),
    'metric': ['', 'auc'],
    'force_row_wise': True,
    'learning_rate': 0.05,
    'max_depth': 5,
    'num_leaves': 2**5,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'min_gain_to_split': 0.01,
    'early_stopping_round': 40,
    'num_iterations': 4000
}

In [ ]:
train_and_save('v2', train_x, train_y, test_x)

In [ ]:
kfold_train_and_save('v2', train_x, train_y, test_x)